# Helping Vancouverites avoid Parking Tickets

More than 3.7 million parking tickets have been issued in Vancouver over the past decade with drivers shelling out an average of <font color=red>$ 32 million dollars</font>  every year.

The average fine of a single ticket is around <font color=blue>$82 dollars</font>. It is a fact that residents need better parking-know how. 

This project aims to combine both parking tickets dataset and parking meters dataset to make customized recommendations  to park to drivers based on different circumstances.

##### I will start by importing all the requisite libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import time 
import folium
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import glob
from PIL import Image

import geopandas as gpd

from shapely.geometry import Point
import matplotlib.colors as colors

from sklearn.cluster import KMeans, MiniBatchKMeans

%matplotlib inline
from shapely.geometry import Polygon, Point, MultiPolygon
import shapefile
import googlemaps
import folium
from sklearn.metrics import silhouette_score

The datasets have been collected from the Open Data Portal provided by the City of Vancouver. The website can be found here https://opendata.vancouver.ca/explore/?q=ope+dat&disjunctive.features&disjunctive.theme&disjunctive.keyword&disjunctive.data-owner&disjunctive.data-team&sort=modified.

There are a total of 4 CSV files organized in order of ascending years starting from 2010 till 2020. 

Since the CSV files were semicolon delimited, I had to specify the appropriate parameter.

In [ ]:
parking_tickets_2010to2013 = pd.read_csv('VancouverParkingTickets/parking-tickets-2010-2013.csv', sep = ';')
parking_tickets_2010to2013

In [ ]:
parking_tickets_2014to2017 = pd.read_csv('VancouverParkingTickets/parking-tickets-2014-2016.csv', sep = ';')
parking_tickets_2014to2017

In [ ]:
parking_tickets_2017to2019 = pd.read_csv('VancouverParkingTickets/parking-tickets-2017-2019-3.csv', sep = ';')
parking_tickets_2017to2019

In [ ]:
parking_tickets_2020 = pd.read_csv('VancouverParkingTickets/parking-tickets.csv', sep = ';')
parking_tickets_2020

 Now, I will read in the parking meters data, which is one complete file. 

In [ ]:
parking_meters = pd.read_csv('VancouverParkingTickets/parking-meters-2.csv', sep = ';')
parking_meters

For ease of computation, I will merge the parking tickets datasets.

In [ ]:
frames = [parking_tickets_2010to2013, parking_tickets_2014to2017, parking_tickets_2017to2019, parking_tickets_2020]

In [ ]:
Total_Parking_Tickets= pd.concat(frames)
Total_Parking_Tickets

There are more than 3.7 million entries here. Let us take a look at what kind of data we are dealing with. 

In [ ]:
Total_Parking_Tickets.info()

BI_ID is the  key field added during import to data warehouse. It is not useful for our purposes.


In [ ]:
Total_Parking_Tickets= Total_Parking_Tickets.drop(['BI_ID'], axis=1)

Let us see if there are any duplicated rows in our data.

In [ ]:
Total_Parking_Tickets.duplicated().value_counts()

About 40% of the rows in our data are duplicate. However, we cannot drop them. Because it is entirely possible that parking enforcement officer or a group of parking enforcement officer issued tickets at the same location on the same date for the same offence. You notice here that the exact time is not given, which could have assisted us in this matter.

I will look at null values now.

In [ ]:
Total_Parking_Tickets.isnull().sum()

This gives us confirmation that the City department has provided us with clean data.

Now, I will repeat the process for the parking meters dataset.

In [ ]:
parking_meters.info()

In [ ]:
parking_meters.duplicated().value_counts()

There are no duplicated meters. What about null values?

In [ ]:
parking_meters.isnull().sum()

We will remove the Rate Misc and Time Misc columns since more than 50% of these columns are null values and also because we will not be using them in our analysis.

In [ ]:
parking_meters = parking_meters.drop(['RATE_MISC', 'TIME_MISC'], axis=1)

In [ ]:
parking_meters['METERHEAD'].value_counts()

Now let us take a look at the remaining null values.

In [ ]:
parking_meters.isnull().sum()

In [ ]:
parking_meters.isnull().sum()

Since they are very few in proportion to the total value counts we can remove the rows.


In [ ]:
parking_meters= parking_meters.dropna()

Let us verify that we have a clean dataset to work with.

In [ ]:
parking_meters.isnull().sum()

#### Addressing Geo locations

Now, since we are dealing with geo-location data, we will ensure this data is in the correct format as per our needs. I will start by modifying the parking_meters dataset first.

In [ ]:
parking_meters['Geom'].value_counts()

I am looking to convert the "Geom" column into 2 separate columns - Latitude and Longitude. Let us look at what type of data is used in this dataset.

In [ ]:
parking_meters.info()

Since, it is a object type column, we can remove the words "type", "Point" and "coordinates" from the column name.

In [ ]:
parking_meters['Geom'] = parking_meters['Geom'].str.replace(r'{"type": "Point", "coordinates":', '')
                                                            
parking_meters
                                                            
                  

Now, we can split the column into 2 parts as mentioned before.

In [ ]:
parking_meters[['Longitude','Latitude']] = parking_meters.Geom.str.split(",",expand=True)
parking_meters

Let us clean the remaining sections of both these new columns and also drop the original column.

In [ ]:
parking_meters['Longitude'] = parking_meters['Longitude'].str.replace(r'{', '')
parking_meters['Longitude'] = parking_meters['Longitude'].str.replace(r'[', '')
parking_meters['Latitude'] = parking_meters['Latitude'].str.replace(r']', '')
parking_meters['Latitude'] = parking_meters['Latitude'].str.replace(r'}', '')


                                                            
                  

In [ ]:
parking_meters= parking_meters.drop(['Geom'], axis=1)

In [ ]:
parking_meters

Now, it is time to work on the Total_Parking_Tickets dataset and create the location columns. We will first create a complete Address column for each parking ticket by concatenating the Block and Street columns and adding the name of the city and country. This is done in accordance to the Google API requirements which we will use later.



In [ ]:
Total_Parking_Tickets['Address'] = Total_Parking_Tickets['Block'].map(str) + " " +  Total_Parking_Tickets['Street'] + " Vancouver " + " Canada "

Also, we drop the original columns now.

In [ ]:
Total_Parking_Tickets= Total_Parking_Tickets.drop(['Block', "Street"], axis=1)

In [ ]:
Total_Parking_Tickets

Let us start by looking at how the parking tickets are clustered in different locations. This means we want to find out which locations are the most ticketed.

In [ ]:
s = Total_Parking_Tickets['Address']
counts = s.value_counts()
percent100 = s.value_counts(normalize=True).mul(100).round(1).astype(str) + '%'
Most_Ticketed_Streets =pd.DataFrame({'Count': counts, 'Percentage of Total': percent100 }).head(20)
Most_Ticketed_Streets.columns = ['Count', 'Percentage of Total']
Most_Ticketed_Streets.index.names = ['Location']
Most_Ticketed_Streets

Let us plot the top 20 addresses here.

In [ ]:
plt.title('Number of Tickets')
plt.ylabel('No. of Violations')
plt.xlabel('Location')
Total_Parking_Tickets['Address'].value_counts().nlargest(20).plot.bar()
positions = (0, 1,2, 3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19)
labels = ("1100 Alberni St", "500 8th Ave W.", "1000 Robson St.","1100 Homer St.","1000 Homer St.","800 Homer St.","1100 Robson St.","1000 Mainland St","2200 4th Ave W.","1000 Cordova St.", "1000 Alberni St.","1100 Hamilton St.", "1900 4th Ave W.","900 Homer St.","1000 Hamilton St.","2100 4th Ave W.","1100 Mainland St","800 Richards St.","800 Seymour St.","2000 4th Ave W.")
plt.xticks(positions, labels)



How many unique addresses are there?

In [ ]:
Total_Parking_Tickets['Address'].nunique()


Let us see count the total values in our dataset.

In [ ]:
Total_Parking_Tickets['Address'].value_counts().sum()

And what number of these tickets are clustered in the top 5000 locations? This number has been derived from trial and error.

In [ ]:
Total_Parking_Tickets['Address'].value_counts().head(5000).sum()


This means that the top 5000 locations accounts for almost 3711580/3490715 ≃ **94%**  of all tickets being issued. Since our project deals with offering options to drivers where they are most likely to be ticketed, we will be using only these locations moving forward. This will help us save valuable processing time and improve the accuracy of our models as well

We will be using Google API to get the geo-locations for our addresses. Ensure you create an account here:https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwjru7W0j_jrAhWFCjQIHYCKBhgQFjAAegQIAxAB&url=https%3A%2F%2Fconsole.developers.google.com%2F&usg=AOvVaw39ieEDI7pzBj4NtuzqS57M and create an API key. Use this key to access this service.

In [ ]:
from googlemaps import Client as GoogleMaps

In [ ]:
api_key = "INSERT API_KEY HERE"


In [ ]:
gmaps = GoogleMaps("INSERT API_KEY HERE")

In [ ]:
Most_Ticketed_Streets['Longitude'] = ""
Most_Ticketed_Streets['Latitude'] = ""

Remember to include a time delay. This is very importnat since the system might crash due to so many requests at once.

In [ ]:
import time
from time import sleep
for x in range(len(new_data2)):
    try:
        time.sleep(2) 
        geocode_result = gmaps.geocode(Most_Ticketed_Streets['index_col'][x])
        Most_Ticketed_Streets['Latitude'][x] = geocode_result[0]['geometry']['location'] ['lat']
        Most_Ticketed_Streets['Longitude'][x] = geocode_result[0]['geometry']['location']['lng']
    except IndexError:
        print("Address was wrong...")
    except Exception as e:\
        print("Unexpected error occurred.", e )
Most_Ticketed_Streets

Then we save the file to a csv for further use and to ensure data is not lost.

In [ ]:
Most_Ticketed_Streets.to_csv('Top 5000 locations.csv')

In [ ]:
Most_Ticketed_Streets = pd.read_csv('VancouverParkingTickets/Top 5000 locations.csv')


Let us look at our file

In [ ]:
Most_Ticketed_Streets




Our dataset does contain some null values. These are the locations Google API could not derive the coordinates for. Let us remove them. 

In [ ]:
Most_Ticketed_Streets = Most_Ticketed_Streets.dropna()

Now that we have the geo locations for both datasets, let us learn more about how data is distributed in other columns. 

I want to find out what is the status of tickets that are issued. 

In [ ]:
Total_Parking_Tickets['Status'].value_counts()


In [ ]:
Status_count  = Total_Parking_Tickets['Status'].value_counts()

plt.figure(figsize=(10,5))
sns.barplot(Status_count.index, Status_count.values, alpha=0.8)
plt.title('Status of Parking Tickets')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Status', fontsize=12)
plt.show()

We can see that around 10% of all tickets that are issued are eventually voided for some reason. What about the reasons these tickets are issued?

In [ ]:
Total_Parking_Tickets['Bylaw'].value_counts()

In [ ]:
Bylaw_count  = Total_Parking_Tickets['Bylaw'].value_counts()

plt.figure(figsize=(10,5))
sns.barplot(Bylaw_count.index, Bylaw_count.values, alpha=0.8)
plt.title('Bylaws Violated as per City of Vancouver')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Bylaw', fontsize=12)
plt.show()

The majority of tickets are issued for violationg Bylaws 2849 and 2952.

Which parts of the city are the parking meters clustered in?

In [ ]:
parking_meters['Geo Local Area'].value_counts()

In [ ]:
Local_Area_count  = parking_meters['Geo Local Area'].value_counts()
plt.figure(figsize=(10,5))
chart =sns.barplot(Local_Area_count.index, Local_Area_count.values, alpha=0.8)
plt.title('Bylaws Violated as per City of Vancouver')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Bylaw', fontsize=12)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.show()

The meters are clustered in downtown mostly, with fairview coming a distant second.

Let us see how the tickets are distributed over the years.

In [ ]:
Yearly_dist = Total_Parking_Tickets.groupby('Year')['Bylaw'].value_counts().sort_index()
Yearly_dist

In [ ]:
sns.boxplot(x = Yearly_dist.index.get_level_values('Year'), y = Yearly_dist).set(
    xlabel='Year', 
    ylabel='Number of Tickets'
)
plt.show()

This gradual increase in the number of tickets issued shows us the increase in number of cars plying on roads in Vancouver, leading to more competition to find that 'valued' parkign spot. Year 2020 is an exception. Covid-19 had an impact on the parking industry as well, and this project was done in September and hence contains data from only the first 8 months of the year.

What about the monthly distribution? Are there some months were tickets tend to be issued more

In [ ]:
Total_Parking_Tickets['Month'] = pd.DatetimeIndex(Total_Parking_Tickets['EntryDate']).month
Total_Parking_Tickets.head()

In [ ]:
Monthly_dist = Total_Parking_Tickets.groupby('Month')['Bylaw'].value_counts().sort_index()
Monthly_dist

In [ ]:
sns.boxplot(x = Monthly_dist.index.get_level_values('Month'), y = Monthly_dist).set(
    xlabel='Year', 
    ylabel='Number of Tickets'
)
plt.show()

In general, the holiday season, at the end of the year and the month of January does lead to a bumper rise in revenue for the City of Vancouver through parking tickets. This might be due to increase to more cars being parked in more regions of the city. It would be great to dwelve on the reasons in coming months. 

Are the meters also located in the same places tickets are clustered? We will be using the foilum library to plot maps in our project. I have learnt the basics of this library from https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/ .Let us make it work!

We start by looking at the overview of the city.

In [ ]:
Vancouver_Map = folium.Map(location = [49.2820, -123.1171], zoom_start=13)
Vancouver_Map

We will start by loooking at a heat map of the most ticketed regions

In [ ]:
from folium import plugins
from folium.plugins import HeatMap


Vancouver_Map =folium.Map(location = [49.2820, -123.1171], zoom_start=13)




Most_Ticketed_Streets['Latitude'] = Most_Ticketed_Streets['Latitude'].astype(float)
Most_Ticketed_Streets['Longitude'] = Most_Ticketed_Streets['Longitude'].astype(float)


heat_df = Most_Ticketed_Streets[['Latitude', 'Longitude']]
heat_df = Most_Ticketed_Streets.dropna(axis=0, subset=['Latitude','Longitude'])


heat_data = [[row['Latitude'],row['Longitude']] for index, row in heat_df.iterrows()]


HeatMap(heat_data).add_to(Vancouver_Map)

Vancouver_Map

This library is working well. But we will need to see the exact locations to provide a precise solution. Let us see the exact locations of our parking meters.

In [ ]:
Vancouver_Map =folium.Map(location = [49.2820, -123.1171], zoom_start=13)

occurences = folium.map.FeatureGroup()


for lat, lng in zip(parking_meters['Latitude'],
                                         parking_meters['Longitude']):
                                         
                                        
    occurences.add_child(
        folium.vector_layers.CircleMarker(
            [lat, lng],
            radius=5, 
            color='green',
            fill=True,
            fill_color='purple',
            fill_opacity=0.6,
            
        )
    )

Vancouver_Map.add_child(occurences)

We do the same for the top 5000 locations in our  Most_Ticketed_Streets dataset.

In [ ]:
Vancouver_Map =folium.Map(location = [49.2820, -123.1171], zoom_start=13)

occurences = folium.map.FeatureGroup()


for lat, lng in zip(Most_Ticketed_Streets['Latitude'],
                                         Most_Ticketed_Streets['Longitude']):
                                         
                                        
    occurences.add_child(
        folium.vector_layers.CircleMarker(
            [lat, lng],
            radius=5, 
            color='black',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6,
            
        )
    )

Vancouver_Map.add_child(occurences)

We can see that both of them are clustered in the same regions.This is a great win for our analysis.

I will be now using these maps to find out what other important differences exist in our meters that we would ultimately recommed to our drivers.

We will start by analyzing which meters accept cash and which accept credit.

In [ ]:
parking_meters['CREDITCARD'].value_counts()

We can see that the majority of them accept cash. Let us map this out. We will use one-hot encoding to make a separate column.

In [ ]:
credit_card = parking_meters[['CREDITCARD','Longitude','Latitude']]

In [ ]:
one_hot1 = pd.get_dummies(credit_card['CREDITCARD'])


In [ ]:
credit_card= pd.concat([credit_card, one_hot1], axis=1)
credit_card

Now we use the colours in such a way that meters which accept cash or credit have different colours. The library we imported for this was import matplotlib.colors as colors

In [ ]:
map_clusters_credit_card = folium.Map(location=[49.2820, -123.1171], zoom_start=4)

# set color scheme for the clusters
x = np.arange(2)
ys = [i + x + (i*x)**2 for i in range(2)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, count in zip(credit_card['Latitude'], credit_card['Longitude'],  
                                            credit_card['No']):
                                        
    label = folium.Popup(str(count) + '- Cluster', parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        tooltip = str(count) + '- Cluster',
        color=rainbow[count-1],
        fill=True,
        fill_color=rainbow[count-2],
        fill_opacity=0.1).add_to(map_clusters_credit_card)
       
map_clusters_credit_card

Repeating the process to show the different types of meter heads.

In [ ]:
parking_meters['METERHEAD'].value_counts()

As you can see here, there are more than 2 types of parking meters, hence we will use an encoding method instead of one-hot encoding

In [ ]:
meter_head = parking_meters[['METERHEAD', 'Latitude', 'Longitude']]

In [ ]:
meter_head["METERHEAD"]=parking_meters["METERHEAD"].astype('category')
meter_head["METERHEAD"] = meter_head["METERHEAD"].cat.codes
meter_head

In [ ]:
map_clusters_meterheads = folium.Map(location=[49.2820, -123.1171], zoom_start=4)
# set color scheme for the clusters
x = np.arange(6)
ys = [i + x + (i*x)**2 for i in range(6)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, cluster in zip(meter_head['Latitude'], meter_head['Longitude'],  
                                            meter_head['METERHEAD']):
                                         
    label = folium.Popup(str(cluster) + '- Cluster',parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        tooltip = str(cluster) + '- Cluster ',
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters_meterheads )


map_clusters_meterheads 





## Clustering Techniques

Now that we know how our datasets are distributed, let us use clustering methods to identify specific clusters in our datasets. I learnt the concept I have applied here from a paid course that you can find here: https://www.coursera.org/learn/clustering-geolocation-data-intelligently-python/home/welcome

We will start by making a new datframe which includes our specific locations.

In [ ]:
X_weighted =Most_Ticketed_Streets.loc[:,['Address','Latitude','Longitude']]

In [ ]:
from shapely.geometry import Point, shape

locs_geometry = [Point(xy) for xy in zip(X_weighted.Longitude,
                                         X_weighted.Latitude)]
crs = {'init': 'epsg:4326'}
# Coordinate Reference Systems, "epsg:4326" is a common projection of WGS84 Latitude/Longitude
locs_gdf = gpd.GeoDataFrame(X_weighted, crs=crs, geometry=locs_geometry)

#### K-Means

K-Means is the most simple of all our clustering methods. Here, you do have to specify the number of clusters. (I used K value=4). K-Means Clustering then finds 4 number of clusters, given a centroid. First it assigns a random point as centroid and then make a cluster. Then it iteratively finds a better centroid and repeats the process until it settles on the given dataset. In this case there are 3 main clusters and 1 outlier. K-Means has the advantage of being more simple than other clustering techniques. However, you need to have the intuition about how many clusters exist in your data, which is difficult especially when dealing with higher dimensional data.

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from ipywidgets import interactive
from collections import defaultdict
Z = np.array(X_weighted[['Longitude', 'Latitude']], dtype= 'float64')
k=4
model= KMeans(n_clusters=k, random_state=17).fit(Z)
class_predictions=model.predict(Z)
X_weighted[f'Cluster_kmeans_new{k}']= class_predictions

Since we are dealing with geo-locational data, we have to be really careful about interpreting the disatnces as pure numbers. You can read more about it here: https://www.thoughtco.com/degree-of-latitude-and-longitude-distance-4070616. Hence I have not used the most commonly used Inertia score here. 

The accuracy metric i have used here is Silhouette Score. It assigns a value of 1 or -1 to each point depending on the point;s closeness to the cluster it was assigned or to some other cluster. The closer the Silhouette Score is to 1, the better the accuracy. This is a much simpler method.

In [ ]:
print(f'K={k}')
print(f'Silhouette Score: {silhouette_score(Z, class_predictions)}')  

In [ ]:
X_weighted['Cluster_kmeans_new4'].value_counts()

Let us map this out.

In [ ]:
import folium
locs_map = folium.Map(location=[49.2827 , -123.1207],
                      zoom_start=9, tiles='cartodbpositron')

feature_ea = folium.FeatureGroup(name='Medium Risk')
feature_pr = folium.FeatureGroup(name='Low Risk')
feature_sr = folium.FeatureGroup(name='High Risk')

for i, v in X_weighted.iterrows():
    
    
    if v['Cluster_kmeans_new4'] == 1:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_ea)
    elif v['Cluster_kmeans_new4'] == 0:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                    
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_pr)
    elif v['Cluster_kmeans_new4'] == 3:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                           
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_sr)

feature_ea.add_to(locs_map)
feature_pr.add_to(locs_map)
feature_sr.add_to(locs_map)
folium.LayerControl(collapsed=False).add_to(locs_map)
locs_map

Let us find out the best k value where the silhouette score is the highest, for our model.

In [ ]:
from tqdm import tqdm
from sklearn.metrics import silhouette_score
best_silhouette, best_k = -1, 0
for k in tqdm(range(2,10)):
    model= KMeans(n_clusters=k, random_state=1).fit(Z)
    class_predictions = model.predict(Z)
    
    curr_silhouette = silhouette_score(Z, class_predictions)
    if curr_silhouette > best_silhouette:
            best_k = k
            best_silhouette = curr_silhouette
print(f'K={k}')
print(f'Silhouette Score: {best_silhouette}')   

As you can notice here, it is true that the higher the number of clusters, then better the Silhouette Score because it get easier for our model to assign a point to a smaller cluster. However, for the ease of undertsanding and classification we will use only 3 clusters- basically labellling each location as High, Medium or Low Risk.
Later, we can try out more number of clusters for a more precise answer.

#### DBSCAN

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) is a more complicated yet precise model since it will look at the density of our locations. It considers a global density for all clusters which means it assumes that all clusters have the same density. Here you do not have to specify number of clusters. Using an epsilon value of 0.01 and min_samples value of 5(hyperparameters). Epsilon value is the maximum distance between 2 points of the same cluster. Minimum samples refers to the number of points surrounding a particular point for that point to be considered a core point. I was able to get a model that projected 3 clusters.


In [ ]:
from sklearn.cluster import DBSCAN
model= DBSCAN(eps=0.01, min_samples=5).fit(Z)
class_predictions = model.labels_
X_weighted["Clusters_DBSCAN"] = class_predictions

Now, here you have a chance to have more outliers. This is inbuilt in this model and do lead to a decrease in your accuracy as can be seen below. On this occassion, this model has very few outliers, which is a great sign. This means almost all of our locations fall in some cluster. 

In [ ]:
print(f'Number of Clusters found: {len(np.unique(class_predictions))}')
print(f'Number of outliers found: {len(class_predictions[class_predictions==-1])}')
print(f'Silhouette ignoring outliers: {silhouette_score(Z[class_predictions!=-1], class_predictions[class_predictions!=-1])}')
no_outliers = np.array([(counter+2)*x if x==-1 else x for counter, x in enumerate(class_predictions)])
print(f'Silhouette outliers as singletons: {silhouette_score(Z, no_outliers)}')

In [ ]:
Let us see the concentration of these clusters.

In [ ]:
X_weighted['Clusters_DBSCAN'].value_counts()

In [ ]:
import folium
locs_map = folium.Map(location=[49.2827 , -123.1207],
                      zoom_start=9, tiles='cartodbpositron')

feature_ea = folium.FeatureGroup(name='Cluster 1 ')
feature_pr = folium.FeatureGroup(name='Cluster 2 ')
feature_sr = folium.FeatureGroup(name='Cluster 3 ')

for i, v in X_weighted.iterrows():
    
    
    if v['Clusters_DBSCAN'] == 0:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_ea)
    elif v['Clusters_DBSCAN'] == 1:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                    
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_pr)
    elif v['Clusters_DBSCAN'] == -1:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                           
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_sr)

feature_ea.add_to(locs_map)
feature_pr.add_to(locs_map)
feature_sr.add_to(locs_map)
folium.LayerControl(collapsed=False).add_to(locs_map)
locs_map

Here, the model has not been able to distinguish the locations as well. If we ignore the outliers, the model does better than K-Means but falls behind if we take the outliers into consideration. 

#### HDBSCAN

Since we are dealing with data that is denser in downtown and very parse in other regions, let us use a method that gives more weightage to density. HDBSCAN (Hierarchical DBSCAN) is a step ahead of the DBSCAN model. It should give us the best accuracy since we know that the tickets are denser in downtown and as we move to the outskirts of the city there are fewer tickets issued. 

In [ ]:
import hdbscan
model = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=2, cluster_selection_epsilon=0.01)
class_predictions = model.fit_predict(Z)
X_weighted["Clusters_HDBSCAN"] = class_predictions

Here again, I have used  similar hyperparameters to get same number of clusters. Remember, we can always change them to get different results as per our specific needs. 

In [ ]:
print(f'Number of clusters found: {len(np.unique(class_predictions))}')
print(f'Number of outliers found: {len(class_predictions[class_predictions==-1])}')
print(f'Silhouette ignoring outliers: {silhouette_score(Z[class_predictions!=-1], class_predictions[class_predictions!=-1])}')
no_outliers = np.array([(counter+2)*x if x==-1 else x for counter, x in enumerate(class_predictions)])
print(f'Silhouette outliers as singletons: {silhouette_score(Z, no_outliers)}')

Very similar results to DBSCAN, but better acccuracy, especially if you consider outliers! Let us plot the distribution here as well.

In [ ]:
X_weighted['Clusters_HDBSCAN'].value_counts()

In [ ]:
import folium
locs_map = folium.Map(location=[49.2827 , -123.1207],
                      zoom_start=9, tiles='cartodbpositron')

feature_ea = folium.FeatureGroup(name='Cluster 1 ')
feature_pr = folium.FeatureGroup(name='Cluster 2 ')
feature_sr = folium.FeatureGroup(name='Cluster 3 ')

for i, v in X_weighted.iterrows():
    
    
    if v['Clusters_HDBSCAN'] == 0:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_ea)
    elif v['Clusters_HDBSCAN'] == 1:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                    
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_pr)
    elif v['Clusters_HDBSCAN'] == -1:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                           
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_sr)

feature_ea.add_to(locs_map)
feature_pr.add_to(locs_map)
feature_sr.add_to(locs_map)
folium.LayerControl(collapsed=False).add_to(locs_map)
locs_map

Here, you can see a larger number of locations are included in the main cluster. We can change the hyperparameters to get more number of clusters that are more precise.

#### Addressing Outliers

When we do provide parking recommendations to customers, we need to be sure about the risk of every location. Hence outliers will have to be grouped in some cluster. A disadvantage with DBCSAN and HDBCAN is they pick out outliers. For or purposes, we want to address the outlier problem and ensure all points are clustered in some way. For this we will use a Hybrid approach, combining our best model (DBSCAN) with K-Nearest Neighbors to assign each outlier the cluster than belongs to its nearest point. 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier= KNeighborsClassifier(n_neighbors=1)
df_train= X_weighted[X_weighted.Clusters_HDBSCAN!=-1]
df_predict= X_weighted[X_weighted.Clusters_HDBSCAN==1]

In [ ]:
X_train = np.array(df_train[['Longitude', 'Latitude']], dtype='float64')
y_train = np.array(df_train['Clusters_HDBSCAN'])
X_predict = np.array(df_predict[['Longitude', 'Latitude']], dtype='float64')

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
X_weighted['Cluster_Hybrid'] = X_weighted['Clusters_HDBSCAN']

In [ ]:
class_predictions = X_weighted.Cluster_Hybrid
print(f'Number of Clusters found: {len(np.unique(class_predictions))}')
print(f'Silhouette Score: {silhouette_score(Z, class_predictions)}')

Overall, this might decrease the accuracy a bit, but it does serve our purpose.

### Plotting the difference between K-Means & Hybrid model

In [ ]:
X_weighted['Cluster_Hybrid'].value_counts().plot.hist(bins=3, alpha=0.4, label='Hybrid')
X_weighted['Cluster_kmeans_new4'].value_counts().plot.hist(bins=3, alpha=0.4, label='Cluster_kmeans(3)')
plt.legend()
plt.title('Comparing Hybrid & K Means Approaches')
plt.xlabel('Cluster_sizes')

Even though all our models had similar accuracies, you can see here that using the hybrid model, there is a huge difference between the clusters. With K-Means, clusters are a bit intertwined. 
In the end, the ultimate decision to use a particular model depends on ease of undersatnding and use. In this case, K-Means is the preferred option which helps explain our project better. 

## Providing a customised solution to our customer

With all that knowledge, we can help provide a customised solution.

Imagine someone is going to 1000 Cordova street from 1000 Hamilton Street. 
We need to plot 3 things here. 

1) The Route

2) The Risk of Being Ticketed

3)The location of specific parking meters which meet their requirements. 

Here the risk of being ticketed is an informed guess, knowing the concentration of parking tickets being issued.

In [ ]:
import folium
locs_map = folium.Map(location=[49.2827 , -123.1207],
                      zoom_start=9, tiles='cartodbpositron')
points_z = [[49.276537,-123.118980], [49.287335,-123.115269]]

feature_ea = folium.FeatureGroup(name='Medium Risk')
feature_pr = folium.FeatureGroup(name='Low Risk')
feature_sr = folium.FeatureGroup(name='High Risk')

for i, v in X_weighted.iterrows():
    
    
    if v['Cluster_kmeans_new4'] == 1:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                            
                            color='#FFBA00',
                            fill_color='#FFBA00',
                            fill=True).add_to(feature_ea)
    elif v['Cluster_kmeans_new4'] == 0:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                    
                            color='#087FBF',
                            fill_color='#087FBF',
                            fill=True).add_to(feature_pr)
    elif v['Cluster_kmeans_new4'] == 3:
        folium.CircleMarker(location=[v['Latitude'], v['Longitude']],
                            radius=1,
                           
                            color='#FF0700',
                            fill_color='#FF0700',
                            fill=True).add_to(feature_sr)

feature_ea.add_to(locs_map)
feature_pr.add_to(locs_map)
feature_sr.add_to(locs_map)
folium.LayerControl(collapsed=False).add_to(locs_map)

parking_points = []
for lat, lng, in zip(parking_meters['Latitude'],
                                         parking_meters['Longitude']):
                                        

        
         
                locs_map.add_child(
                folium.Marker([lat, lng], popup='Timberline Lodge',
                               icon=folium.Icon(icon="credit-card", prefix="fa")
             )
    )                               
# add markers 'z'
for each in points_z:  
    locs_map.add_child(folium.CircleMarker(location=each,
    fill='true',
    radius = 6,
    popup= 'Bye',
    fill_color='yellow',
    color = 'clear',
    fill_opacity=1))
    
for each in points_z:  
    folium.Marker(each).add_to(locs_map)

folium.PolyLine(points_z, color="black", weight=2.5, opacity=1).add_to(locs_map)


locs_map

As you can see, the driver has multiple options to choose from and can go to the parking spot nearest to them which is available. 

This project gives us endless possibilities of additions/customizations to pour earlier findings. We can specify the time and select only those parking meters that are open during that time. We can choose the cheapest option as well. Another addition would be the time of the year, day which will make it even more realistic.

Following these footsteps, we can create new examples as well. I can team up with a web developer to make a full-fledged application that allows users to make real time decisions. We can also include the weather updates, traffic movements and even off-street parking options in our analysis.
There is huge opportunity in this space to make further updates and use this information for the benefit of the general public. I thank the Education Team and my peers who provided much needed support for this project.
